# Obtención de los DataFrames

In [ ]:
# Importar todo lo necesario

import numpy as np
import pandas as pd
import scipy as sp
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

### DataFrame de MovieLens

Se obtiene el DataFrame del conjunto de datos de MovieLens y se comprueba rápidamente es compatible con el modelo de LightFM.

In [ ]:
# Obtención del dataframe de ml_data
ml_data_df = pd.read_csv('/Users/raulnegro/Google Drive/GitHub/SistemaRecomendacionTFG/data/ml_data.csv', 
    delim_whitespace=True, names=['Id Usuario','Id Película','Valoración','Fecha'])

# Descomentar para comprobar que el dataframe se ha obtenido correctamente
#ml_data_df

# Obtención del dataframe de ml_items
ml_items_df = pd.read_csv('/Users/raulnegro/Google Drive/GitHub/SistemaRecomendacionTFG/data/ml_items.csv',
    sep='|', names=['Id Película','Título','Fecha de estreno','Fecha DVD','iMDB','Género desconocido','Acción','Aventura','Animación','Infantil','Comedia', 'Crimen','Docuemntal','Drama','Fantasía','Cine negro','Horror','Musical','Misterio','Romance','Ciencia ficción','Thriller','Bélico','Western'],
    encoding='latin-1')

# Descomentar para comprobar que el dataframe se ha obtenido correctamente
#ml_items_df

# Junto los dos dataframes
ml_df = pd.merge(ml_data_df, ml_items_df.drop_duplicates(['Id Película']), on="Id Película", how="left")

# Descomentar para comprobar que el dataframe se ha obtenido correctamente
#ml_df.head()

ml_df_2 = ml_df.groupby('Id Usuario').agg(lambda x: list(x))
ml_df_2.head()


In [ ]:
# Comprobación de la precisión con LightFM
model = LightFM(loss='warp')
model.fit(sp.sparse.csr_matrix(ml_data_df.values), epochs=30, num_threads=2)
test_precision = precision_at_k(model, sp.sparse.csr_matrix(ml_data_df.values), k=5).mean()
test_precision

In [ ]:
scores = model.predict([3], np.arange(4))
top_items = ml_df['Título'][np.argsort(-scores)]
print(top_items)